In [2]:
import sys
assert sys.version_info >= (3,5)

import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

from tensorflow.python.compiler.mlcompute import mlcompute
mlcompute.set_mlc_device(device_name='gpu') # Available options are 'cpu', 'gpu', and ‘any'.

import numpy as np
import os

np.random.seed(211122)
tf.random.set_seed(211122)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = '.'
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension='png',resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure",fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 텐서와 연산 

## 텐서

In [4]:
tf.constant([[1.,2.,3.],[4.,5.,6.]]) #행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
tf.constant(42) #스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [6]:
t =tf.constant([[1.,2.,3.],[4.,5.,6.]]) 
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [7]:
t.shape

TensorShape([2, 3])

In [8]:
t.dtype

tf.float32

## 인덱싱 

In [9]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [14]:
t[...,1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

In [19]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

## 연산 

In [20]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [21]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [23]:
t @ tf.transpose(t) # @ : 행렬곱셈

2021-11-22 15:57:24.723159: I tensorflow/compiler/tf2mlcompute/utils/mlc_utils.cc:175] Eager mode uses the CPU. Switching to the CPU.


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [27]:
tf.transpose(t) @ t

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[17., 22., 27.],
       [22., 29., 36.],
       [27., 36., 45.]], dtype=float32)>

In [25]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [28]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 4.],
       [2., 5.],
       [3., 6.]], dtype=float32)>

## keras.backend 사용하기 

In [29]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

## 넘파이 변환 

In [30]:
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [31]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [32]:
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [33]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## 타입 변환 

In [36]:
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [42]:
try:
    tf.constant(2.0) + tf.constant(40.,dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [43]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)
#type 변환 함수 cast

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 변수 

In [44]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])

In [46]:
v.assign(2*v) #assign 함수로 바로 변경 가능

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

## 자동 미분을 사용하여 그레디언트 계산하기

다음의 도함수 계산할 수 있음. (w1,w2) = (5,3)에서 도함수의 값은 (36,10)임

In [52]:
def f(w1,w2):
    return 3*w1**2 + 2*w1*w2

직접 도함수 다 계산하는 것은 불가능하므로, 각 파라미터가 바뀔 떄마다 함수의 출력이 얼마나 변하는지 측정하여 도함수의 근삿값을 계산해보자

In [53]:
w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1,w2)) / eps

36.000003007075065

In [56]:
(f(w1, w2 + eps) - f(w1,w2)) / eps

10.000000003174137

대규모 신경망에선 적용하기 힘드므로, 자동미분을 사용해보자
tf.GradientTape 블럭을 만들어 해당 변수와 관련된 모든 연산을 자동으로 기록.
이 tape에 두 변수에 대한 z의 그레디언트를 요청

In [57]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)

gradients = tape.gradient(z, [w1,w2])

In [58]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [60]:
with tf.GradientTape() as tape:
    z = f(w1,w2)
    
dz_dw1 = tape.gradient(z,w1)
try:
    dz_dw2 = tape.gradient(z,w2)
except RuntimeError as ex:
    print(ex)

A non-persistent GradientTape can only be used tocompute one set of gradients (or jacobians)


In [66]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1,w2)

dz_dw1 = tape.gradient(z,[[w1,w2]])
dz_dw2 = tape.gradient(z,w2)
del tape #persistent=True로 지속가능하게 만들었으면, 리소스 지워야함

In [67]:
dz_dw1

[[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]]

In [68]:
w1, w2, w3 = tf.Variable(5.), tf.Variable(3.), tf.Variable(6.)

In [69]:
with tf.GradientTape() as tape:
    z = 3*w1**2 + 2*w1*w2 + 3*w3**2
    
dz = tape.gradient(z, [w1,w2,w3])

In [70]:
dz

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=36.0>]